航班数据的处理和简单分析

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

from datetime import datetime

from IPython.display import display

%matplotlib inline

sns.set(font="simhei")

## 读取数据

读取原始数据，原始文件名是中文，手动改成了英文。

In [3]:
train_f = pd.read_csv('input/train/raw/fight-201505-201705.csv', encoding='gb2312', low_memory=False)

display(train_f.head())

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常


In [4]:
train_f.columns = ['Departure', 'Destination', 'FLTNo', 'PDepartureTime', 'PArrivalTime', 'ADepartureTime', 'AArrivalTime', 'Id', 'Cancel']

train_f.head()

,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,Cancel
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常


## 减少内存占用

默认int和float都是64位的，我们可以适当使用更小的数据结构来存储。Cancel默认是object，它其实只有两个值，也可以转换。

In [5]:
train_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7518638 entries, 0 to 7518637
Data columns (total 9 columns):
Departure         object
Destination       object
FLTNo             object
PDepartureTime    int64
PArrivalTime      int64
ADepartureTime    float64
AArrivalTime      float64
Id                float64
Cancel            object
dtypes: float64(3), int64(2), object(4)
memory usage: 516.3+ MB


查看uint32，确认它能够保存我们的时间戳

In [6]:
import numpy as np
int_types = ["uint32", "uint16"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint32
---------------------------------------------------------------
min = 0
max = 4294967295
---------------------------------------------------------------

Machine parameters for uint16
---------------------------------------------------------------
min = 0
max = 65535
---------------------------------------------------------------



### PDepartureTime、PArrivalTime

PDepartureTime、PArrivalTime转换为uint32

In [7]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

print(mem_usage(train_f['PDepartureTime']))
train_f['PDepartureTime'] = pd.to_numeric(train_f['PDepartureTime'], downcast='unsigned')
print(mem_usage(train_f['PDepartureTime']))

57.36 MB
28.68 MB


In [8]:
print(mem_usage(train_f['PArrivalTime']))
train_f['PArrivalTime'] = pd.to_numeric(train_f['PArrivalTime'], downcast='unsigned')
print(mem_usage(train_f['PArrivalTime']))

57.36 MB
28.68 MB


### ADepartureTime、AArrivalTime

ADepartureTime、AArrivalTime都有Null，需要先填0，然后再转成uint32。其实它俩可以转成float32，与unit32占用的内存是一样的，但后面的Id可以使用uint16（Null值填0），所以这里也这样处理。

In [9]:
print(mem_usage(train_f['ADepartureTime']))
train_f['ADepartureTime'] = train_f['ADepartureTime'].fillna(0).astype(int)

train_f['ADepartureTime'] = pd.to_numeric(train_f['ADepartureTime'], downcast='unsigned')
print(mem_usage(train_f['ADepartureTime']))

57.36 MB
28.68 MB


In [10]:
print(mem_usage(train_f['AArrivalTime']))
train_f['AArrivalTime'] = train_f['AArrivalTime'].fillna(0).astype(int)

train_f['AArrivalTime'] = pd.to_numeric(train_f['AArrivalTime'], downcast='unsigned')
print(mem_usage(train_f['AArrivalTime']))

57.36 MB
28.68 MB


### Id

Null填零，使用uint16类型

In [11]:
print(mem_usage(train_f['Id']))
train_f['Id'] = train_f['Id'].fillna(0).astype(int)

train_f['Id'] = pd.to_numeric(train_f['Id'], downcast='unsigned')
print(mem_usage(train_f['Id']))

57.36 MB
14.34 MB


### Cancel

Cancel是二值标签，转uint8就OK了。

In [12]:
print(mem_usage(train_f['Cancel']))
train_f['Cancel'] = train_f['Cancel'].map({'正常': 0, '取消': 1})
print(mem_usage(train_f['Cancel']))
train_f['Cancel'] = pd.to_numeric(train_f['Cancel'], downcast='unsigned')
print(mem_usage(train_f['Cancel']))

616.65 MB
57.36 MB
7.17 MB


In [13]:
train_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7518638 entries, 0 to 7518637
Data columns (total 9 columns):
Departure         object
Destination       object
FLTNo             object
PDepartureTime    uint32
PArrivalTime      uint32
ADepartureTime    uint32
AArrivalTime      uint32
Id                uint16
Cancel            uint8
dtypes: object(3), uint16(1), uint32(4), uint8(1)
memory usage: 308.3+ MB


内存减少的效果还是比较明显的， 减少了200M+的内存占用。

## 时间戳

我们以最大的计划起飞时间为样例，来探索一下时间戳有关的问题。

In [14]:
train_f.loc[train_f['PDepartureTime'] == train_f['PDepartureTime'].max(), :].head()

,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,Cancel
4906881,DYG,XIY,MU2432,1496246100,1496252100,1496246100,1496250180,1511,0
4924435,CSX,NKG,MU2746,1496246100,1496251800,1496245980,1496250300,2110,0
4928507,URC,AKU,CZ6920,1496246100,1496250900,1496246580,1496250120,2902,0
4933127,JIQ,CTU,3U8564,1496246100,1496251200,1496245440,1496248920,1934,0
4970563,CSX,CKG,PN6232,1496246100,1496251500,1496245620,1496250120,198,0


先把第一行的计划起飞时间和实际起飞时间拿出来，组成一个新的DataFrame。

In [15]:
data = {'PDepartureTime': [1496246100], 'PArrivalTime': [1496252100]}
df = pd.DataFrame(data, columns=data.keys())
df

,PArrivalTime,PDepartureTime
0,1496252100,1496246100


用Python的datetime库先转换成datetime.datetime对象，然后按照指定格式转成字符串。

In [16]:
def strftime(row):
    p_time = datetime.fromtimestamp(row['PDepartureTime']).strftime('%Y-%m-%d %H:%M:%S')
    a_time = datetime.fromtimestamp(row['PArrivalTime']).strftime('%Y-%m-%d %H:%M:%S')
    return [p_time, a_time]

df[['PDTime', 'PATime']] = df.apply(strftime, axis=1)
df.head()

,PArrivalTime,PDepartureTime,PDTime,PATime
0,1496252100,1496246100,2017-05-31 23:55:00,2017-06-01 01:35:00


然后用Pandas的to_datatime函数，转换成pandas._libs.tslib.Timestamp对象。

In [17]:
df['PDTime2'] = pd.to_datetime(df['PDepartureTime'], unit='s')
df['PATime2'] = pd.to_datetime(df['PArrivalTime'], unit='s')
df

,PArrivalTime,PDepartureTime,PDTime,PATime,PDTime2,PATime2
0,1496252100,1496246100,2017-05-31 23:55:00,2017-06-01 01:35:00,2017-05-31 15:55:00,2017-05-31 17:35:00


可以看出Python的datetime转换的结果和Pandas的to_datetime不一致。而且Python的datetime库的结果是正确的。

这是因为datetime考虑了时区分布，而to_datetime默认是标准时间（UTC)。使用to_datetime时，用如下方法可以指定时区：

In [18]:
df['PDTime3'] = pd.to_datetime(df['PDepartureTime'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
df['PATime3'] = pd.to_datetime(df['PArrivalTime'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
df

,PArrivalTime,PDepartureTime,PDTime,PATime,PDTime2,PATime2,PDTime3,PATime3
0,1496252100,1496246100,2017-05-31 23:55:00,2017-06-01 01:35:00,2017-05-31 15:55:00,2017-05-31 17:35:00,2017-05-31 23:55:00+08:00,2017-06-01 01:35:00+08:00


可以发现，to_datetime指定了时区后的结果与Python的datetime库的结果相同了。

如果要想再讲PDTime、PDTime3转换成uninx时间戳的形式，可以使用如下方法：

In [19]:
print('PDTime to unix timestamp:', int(datetime.strptime(df.loc[0, 'PDTime'], '%Y-%m-%d %H:%M:%S').strftime("%s")))

print('PDTime3 to unix timestamp:', (df['PDTime3'].astype(int) // 10 ** 9).loc[0])

print('PDTime3 to datetime.datetime to uninx timestamp:', int(df.loc[0, 'PDTime3'].to_pydatetime().strftime("%s")))

PDTime to unix timestamp: 1496246100
PDTime3 to unix timestamp: 1496246100
PDTime3 to datetime.datetime to uninx timestamp: 1496246100


总之，我们确定了航班数据中的时间戳为北京时间，而如果使用Pandas的to_datetime的时候需要考虑时区的问题，否则时间转换就出问题了。

## 空值

Departure、Destination、FLTNo、PDepartureTime、PArrivalTime、Cancle都没有Null。

### ADepartureTime

ADepartureTime有很多Null值，可以发现其中绝大部门都是因为航班取消了，这也能够理解，航班取消了自然就不会有出发时间了。这种继续保持NaN值就好，后面这个特征肯定会被转换成其他特征而拿掉的。

In [20]:
print('ADepartureTime NaN: ', train_f[train_f['ADepartureTime'] == 0].size)

train_f[train_f['ADepartureTime'] == 0].head()

ADepartureTime NaN:  2912490


,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,Cancel
5,DLC,NNG,ZH953Z,1452385800,1452401700,0,0,6,1
6,HGH,SZX,CZ6327,1452591900,1452600900,0,0,0,1
55,HAK,SZX,HU7013,1452032700,1452037500,0,0,55,1
78,WUX,SZX,MF1094,1452855300,1452864900,0,0,0,1
98,CAN,URC,CZ6884,1453976100,1453996200,0,0,0,1


而下面这些航班没有取消，却没有实际出发时间的就很难弄了。

In [21]:
print('ADepartureTime NaN: ', train_f[(train_f['ADepartureTime']==0) & (train_f['Cancel']!=1)].size)

train_f[(train_f['ADepartureTime']==0) & (train_f['Cancel']!=1)].head()

ADepartureTime NaN:  15444


,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,Cancel
183,HGH,CKG,OQ2380,1453606500,1453616400,0,1453607040,176,0
2767,FOC,PVG,MU5506,1452815700,1452820500,0,1452821940,1239,0
13891,LZH,CTU,EU2202,1453044600,1453050900,0,1453052400,1559,0
15737,KWE,DLC,HU762Z,1451865600,1451883000,0,1451881200,498,0
17352,TCZ,KMG,KY8288,1453020000,1453023600,0,1453023900,168,0


那么我就看航班到达晚点了多少吧，姑且认为到达晚点了多少出发就晚点多少。

In [22]:
# 根据到达晚点的时间 填补 实际出发时间
train_f['ADelay'] = train_f['AArrivalTime'] - train_f['PArrivalTime']

cond = (train_f['ADepartureTime']==0) & (train_f['Cancel']!=1)
train_f.loc[cond, 'ADepartureTime'] = train_f.loc[cond, 'PDepartureTime'] + train_f.loc[cond, 'ADelay']
train_f.drop('ADelay', axis=1, inplace=True)

OK，我们再确认一下剩余的ADepartureTime为空的记录都是已取消的航班。

In [23]:
train_f[(train_f['ADepartureTime']==0) & (train_f['Cancel']!=1)].head()

,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,Cancel


### AArrivalTime

AArrivalTime的Null值也不少，幸运的是，AArrivalTime为Null的航班全部都取消了，我们无需对Null做处理。

In [24]:
print('AArrivalTime NaN: ', train_f[train_f['AArrivalTime']==0].size)

print('AArrivalTime NaN and not Cancel: ', train_f[(train_f['AArrivalTime']==0) & (train_f['Cancel']!=1)].size)


AArrivalTime NaN:  2908188
AArrivalTime NaN and not Cancel:  0


### Id

Id为Null且航班未取消的有27w+，真头疼！

从数据说明和之前的认识中，我已经知道每天同一架飞机的Id是一样的，因此如果要填充Id，就需要根据航班号、出发、到达机场找到前序、后续航班，拿前序、后续航班的Id进行填充。

但这个数据量太大，具体实现要想想办法。

In [25]:
print('Id NaN: ', train_f[(train_f['Id']==0) & (train_f['Cancel']!=1)].size)

train_f[(train_f['Id']==0) & (train_f['Cancel']!=1)].head()

Id NaN:  246249


,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,Cancel
2720,DNH,LHW,MU7582,1452936900,1452942600,1452935940,1452940440,0,0
9012,JHG,CKG,PN6216A,1454255400,1454265900,1454259780,1454269320,0,0
15058,DNH,XIY,GS7680A,1454002200,1454013300,1454002800,1454013420,0,0
15792,CKG,KWE,CZ3641A,1452362400,1452365100,1452368640,1452371640,0,0
16320,CAN,HAK,CZ6778A,1453479300,1453483500,1453480440,1453484040,0,0


## 前序、后序

探索前序、后续航班之间的关系。

先将计划出发时间分割成日期、时间，方便对每天的航班进行分析。

In [26]:
# 执行时间较长，可以喝杯茶、撒泡尿
train_f[['PDDate', 'PDTime']] = pd.to_datetime(train_f['PDepartureTime'], unit='s')\
.dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')\
.dt.strftime('%Y-%m-%d %H:%M:%S').str.split(' ', expand=True)

train_f[train_f['PDepartureTime']==train_f['PDepartureTime'].max()].head()

,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,Cancel,PDDate,PDTime
4906881,DYG,XIY,MU2432,1496246100,1496252100,1496246100,1496250180,1511,0,2017-05-31,23:55:00
4924435,CSX,NKG,MU2746,1496246100,1496251800,1496245980,1496250300,2110,0,2017-05-31,23:55:00
4928507,URC,AKU,CZ6920,1496246100,1496250900,1496246580,1496250120,2902,0,2017-05-31,23:55:00
4933127,JIQ,CTU,3U8564,1496246100,1496251200,1496245440,1496248920,1934,0,2017-05-31,23:55:00
4970563,CSX,CKG,PN6232,1496246100,1496251500,1496245620,1496250120,198,0,2017-05-31,23:55:00
